### Change Log
---
Checking saving and loading

In [1]:
import sys
sys.path.append('/data/extensibleFramework/')

In [2]:
from extensibleFramework.src.models import convolution_neural_network
from extensibleFramework.src.models import recurrent_nn_with_attention
from extensibleFramework.src.models import feed_forward_network
from extensibleFramework.src.models import extra_layers
from extensibleFramework.src.utils import saving_and_loading

In [3]:
import chakin

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from torchtext import data
import nltk
import json
from torchtext import vocab
from tqdm import tqdm
import torch
import random
import re
import pandas as pd
import os
import numpy as np
import sys
import random
import tarfile
import urllib
from torchtext import data
import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
data_frame = pd.read_csv("labeledTrainData.tsv",sep="\t")
data_frame = data_frame[['review', 'sentiment']]
data_block  = data_frame.values

In [6]:
SEED = 1
split = 0.80

In [7]:
random.shuffle(data_block)

# train_file = open('train.json', 'w')
# test_file = open('test.json', 'w')
# for i in  range(0,int(len(data_block)*split)):
#     train_file.write(str(json.dumps({'review' : data_block[i][0], 'label' : data_block[i][1]}))+"\n")
# for i in  range(int(len(data_block)*split),len(data_block)):
#     test_file.write(str(json.dumps({'review' : data_block[i][0], 'label' : data_block[i][1]}))+"\n")

In [8]:
def tokenize(sentiments):
    
    return nltk.tokenize.word_tokenize(sentiments)
# def pad_to_equal(x):
#     if len(x) < 61:
#         return x + ['<pad>' for i in range(0, 61 - len(x))]
#     else:
#         return x[:61]
def to_categorical(x):
    x = int(x)
    if x == 1:
        return [0,1]
    if x == 0:
        return [1,0]

In [9]:
SENTIMENT = data.Field(sequential=True , tokenize=tokenize, use_vocab = True, lower=True,batch_first=True)
LABEL = data.Field(is_target=True,use_vocab = False, sequential=False, preprocessing = to_categorical)
fields = {'sentiment': ('sentiment', SENTIMENT), 'label': ('label', LABEL)}
train_data , test_data = data.TabularDataset.splits(
                            path = '',
                            train = 'train.json',
                            test = 'test.json',
                            format = 'json',
                            fields = fields)


In [10]:
# chakin.search(lang='English')

In [11]:
# chakin.download(number=12, save_dir = "./")

In [12]:
# !unzip glove.6B.zip

In [13]:
vec = vocab.Vectors(name = "glove.6B.100d.txt",cache = "./")

In [14]:
SENTIMENT.build_vocab(train_data, test_data, max_size=400000, vectors=vec)

In [15]:
train_iter, test_iter = data.Iterator.splits(
        (train_data, test_data), sort_key=lambda x: len(x.sentiment),
        batch_sizes=(32,32), device=device)

In [16]:
for batch in test_iter:
    feature, target = batch.sentiment, batch.label
    print(feature.data.shape, target.data.shape)
    break

torch.Size([32, 46]) torch.Size([32, 2])


In [17]:
sentiment_vocab = SENTIMENT.vocab

In [18]:
sentiment_vocab.vectors.shape

torch.Size([118522, 100])

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class ensemble_model(nn.Module):
    def __init__(self, vocab_size, embed_dim, class_num, \
                 out_channel_num, kernel_size, dropout, input_size, hidden_size,\
                 batch_size, n_layers=1, rnn_unit="GRU",weight=None, weight_is_trainable = False):
        super(ensemble_model, self).__init__() 
        self.cnn = convolution_neural_network.cnn_text(vocab_size, embed_dim, class_num, out_channel_num, \
                                                       kernel_size, dropout,weight=None, weight_is_trainable = weight_is_trainable)
        self.rnn_attention = recurrent_nn_with_attention.RNNAttentionModel(output_size=class_num, hidden_size=hidden_size,\
                                                                          vocab_size=vocab_size, embedding_length=embed_dim, \
                                                                          batch_size=batch_size, device = device,\
                                                                          weight=None, weight_is_trainable = weight_is_trainable)
        self.merge_layer = extra_layers.MergeAndFlattern(mode="CONCAT",batch_size=batch_size)
        self.ffn = feed_forward_network.ffn(activation = "Relu", num_layer = 2 , input_size = 200 , output_size = 2, perceptron_per_layer = [100,50] , dropout = [0.2,0.2])
    
    def forward(self, x):
        cnn_output = self.cnn(x)
        rnnAttention_output = self.rnn_attention(x)
        merge_layer_output = self.merge_layer(cnn_output,rnnAttention_output )
        final_output = self.ffn(merge_layer_output)
        
#         print("cnn_output : ",cnn_output.shape)
#         print("rnn_input : ",rnnAttention_output.shape)
#         print("merge_layer_output : ", merge_layer_output.shape)
#         print("final_output : ", final_output.shape)
        return final_output

In [20]:
input_size = 10
hidden_size = 128
batch_size = 32
n_layers = 1
vocab_size = len(SENTIMENT.vocab)
embed_dim  = 100
class_num =  100
out_channel_num = 8
kernel_sizes = [3,4,5]
dropout = 0.2

EM  = ensemble_model(vocab_size=vocab_size, embed_dim=embed_dim, class_num=class_num,out_channel_num=out_channel_num,\
                    kernel_size=kernel_sizes, dropout=dropout, input_size = input_size, hidden_size=hidden_size,\
                    batch_size = batch_size, n_layers = n_layers, rnn_unit = "GRU",weight=sentiment_vocab.vectors, \
                     weight_is_trainable = True)
EM  = EM.to(device)


In [21]:
cnn_input = torch.Tensor(np.random.randint(5, size=[8, 10])).long().to(device)
rnn_input = torch.Tensor(np.random.random([8, 10])).long().to(device) #batch_size, input_size

In [22]:
# EM(rnn_input)

In [23]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded_preds = torch.argmax(preds, dim=1)
    correct = (rounded_preds == torch.argmax(y, dim=1)).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [24]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        feature, target = batch.sentiment, batch.label
        optimizer.zero_grad()
        predictions = model(feature.to(device))            
        loss = criterion(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        loss.backward()
        optimizer.step()
        acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return model, epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
def test_accuracy_calculator(model, test_iterator):
    epoch_acc = 0
    for batch in test_iterator:
        if batch.sentiment.shape[0] ==  32:
            feature, target = batch.sentiment, batch.label
            predictions = model(feature.to(device))            
            acc = binary_accuracy(predictions.type(torch.FloatTensor), target.type(torch.FloatTensor))
            epoch_acc += acc.item()
    return  epoch_acc / len(test_iterator)

In [26]:
optimizer = torch.optim.SGD(EM.parameters(), lr=0.1,momentum=0.9)
criterion = nn.MSELoss()
criterion = criterion.to(device)

In [27]:
# epochs  = 100
# log_interval = 1
# loss = []
# accuracy = []
# test_accuracy = []
# for i in tqdm(range(epochs)):
#     model, epoch_loss, epoch_acc = train(EM, train_iter, optimizer, criterion)
#     test_acc = test_accuracy_calculator(model, test_iter)
#     accuracy.append(epoch_acc)
#     loss.append(epoch_loss)
#     test_accuracy.append(test_acc)
#     print(epoch_acc,test_acc,epoch_loss)

In [28]:
SAL = saving_and_loading.objectManager()

In [29]:
SAL.saver(EM, "./EM.ckpt")

++++++++ Model dumped with following parameters ++++++++
--------------------------------------------------------
cnn.embed.weight 	 torch.Size([118522, 100])
cnn.convs1.0.weight 	 torch.Size([8, 1, 3, 3])
cnn.convs1.0.bias 	 torch.Size([8])
cnn.convs1.1.weight 	 torch.Size([8, 1, 4, 4])
cnn.convs1.1.bias 	 torch.Size([8])
cnn.convs1.2.weight 	 torch.Size([8, 1, 5, 5])
cnn.convs1.2.bias 	 torch.Size([8])
cnn.fc1.weight 	 torch.Size([100, 24])
cnn.fc1.bias 	 torch.Size([100])
rnn_attention.word_embeddings.weight 	 torch.Size([118522, 100])
rnn_attention.lstm.weight_ih_l0 	 torch.Size([512, 100])
rnn_attention.lstm.weight_hh_l0 	 torch.Size([512, 128])
rnn_attention.lstm.bias_ih_l0 	 torch.Size([512])
rnn_attention.lstm.bias_hh_l0 	 torch.Size([512])
rnn_attention.label.weight 	 torch.Size([100, 128])
rnn_attention.label.bias 	 torch.Size([100])
ffn.network.input.weight 	 torch.Size([100, 200])
ffn.network.input.bias 	 torch.Size([100])
ffn.network.Linear0.weight 	 torch.Size([50, 100])


In [30]:
EM = SAL.loader("./EM.ckpt")